# **AARON'S NIGHTMARE EQUATIONS**

## **Install**

### Firedrake

In [1]:
try:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh" -O "/tmp/firedrake-install.sh"
    !bash "/tmp/firedrake-install.sh"
    from firedrake import *  # noqa: F401
except:
    from firedrake import *  # noqa: F401

--2025-11-10 22:03:13--  https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.110.153, 185.199.109.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4767 (4.7K) [application/x-sh]
Saving to: ‘/tmp/firedrake-install.sh’

/tmp/firedrake-inst 100%[===================>]   4.66K  --.-KB/s    in 0s      

2025-11-10 22:03:13 (38.6 MB/s) - ‘/tmp/firedrake-install.sh’ saved [4767/4767]

+ INSTALL_PREFIX=/usr/local
++ awk -F/ '{print NF-1}'
++ echo /usr/local
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FIREDRAKE_INSTALLED=/usr/local/share/fem-on-colab/firedrake.installed
+ [[ ! -f /usr/local/share/fem-on-colab/firedrake.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/1f62

### Irksome

In [2]:
try:
    !python3 -m pip install --no-dependencies git+https://github.com/firedrakeproject/Irksome.git
    from irksome import *  # noqa: F401
except:
    from irksome import *  # noqa: F401

  Cloning https://github.com/firedrakeproject/Irksome.git to /tmp/pip-req-build-5y7tb_n0
  Running command git clone --filter=blob:none --quiet https://github.com/firedrakeproject/Irksome.git /tmp/pip-req-build-5y7tb_n0
  Resolved https://github.com/firedrakeproject/Irksome.git to commit 055fb310b77a19ef3e6f3b6d13a64c85b0041828
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for IRKsome: filename=irksome-0.0.1-py3-none-any.whl size=61831 sha256=49b38ffe99c546d5f281bc65266f2a5085663f3777b3728e453017be0f2a3b28
  Stored in directory: /tmp/pip-ephem-wheel-cache-kqviio09/wheels/d2/9a/b0/fa0519647bfe73fa482134dfb68f1e94bba6822b7c0db00983
Successfully built IRKsome


### Other

In [3]:
from pathlib import Path

## **General functions**

### Aaron's nightmare Stefan–Maxwell equations: Energy/Entropy

In [94]:
def stefan_maxwell(
    Nspec:      int = 2,
    Nx:         int = 24,
    deg:        int = 1,
    vdeg:       int = 2,
    time_deg:   int = 1,
    Nt:         int = 24,
    dt:         float = 1e-9,
    Kval:       float = 1.0e-2,
    nu:         float = 1.0e-3,
    scheme:     str = "gauss",
    output_dir: str = "output/stefan_maxwell/",
    write_qois: bool = False,
    write_vtk:  bool = False,
):
    """
    Energy- and entropy-preserving Stefan-Maxwell scheme.

    - Unknowns (time-continuous): (rho_i), u, (rho s)
    - Auxiliary (time-discontinuous): (mu_i), p, theta, m

    Returns
    - Dictionary: {"time": [...], "energy": [...], "entropy": [...]}
    """
    # Ensure output directory exists
    out_path = Path(output_dir)
    out_path.mkdir(parents=True, exist_ok=True)

    # Convert parameters to UFL objects
    K_c = Constant(Kval)
    nu_c = Constant(nu)
    dt_c = Constant(dt)

    # Mesh and coordinate (2D periodic box)
    mesh = PeriodicUnitSquareMesh(Nx, Nx)
    x, y = SpatialCoordinate(mesh)

    # Function spaces
    S = FunctionSpace(mesh, "CG", deg)  # Scalar
    V = VectorFunctionSpace(mesh, "CG", vdeg)  # Vector
    Z = MixedFunctionSpace(([S]*Nspec) + [V, S] + ([S]*Nspec) + [S, S, V])  # Mixed space: (rho_1...rho_N, u, rho_s, mu_1...mu_N, p, theta, m)

    # Solution functions
    z = Function(Z, name="state")

    z_split = split(z)
    rho = z_split[0:Nspec]; u = z_split[Nspec]; rho_s = z_split[Nspec+1]; mu = z_split[(Nspec+2):(2*Nspec+2)]; p = z_split[2*Nspec+2]; theta = z_split[2*Nspec+3]; m = z_split[2*Nspec+4]

    z_out = z.subfunctions
    rho_out = z_out[0:Nspec]; u_out = z_out[Nspec]; rho_s_out = z_out[Nspec+1]; mu_out = z_out[(Nspec+2):(2*Nspec+2)]; p_out = z_out[2*Nspec+2]; theta_out = z_out[2*Nspec+3]; m_out = z_out[2*Nspec+4]

    # Split tests (UFL)
    tests = TestFunctions(Z)
    psi = tests[0:Nspec]; v = tests[Nspec]; omega = tests[Nspec+1]; zeta = tests[(Nspec+2):(2*Nspec+2)]; q = tests[2*Nspec+2]; gamma = tests[2*Nspec+3]; w = tests[2*Nspec+4]

    # Helpers for rho
    rho_tot = sum(rho)
    sqrt_rho = sqrt(rho_tot)

    # Specific volumes and K, C_i
    V_i = [Constant(1.0) for _ in range(Nspec)]

    # Free energy density (simple ideal mixture without gradient terms)
    rho_F = sum([rho[i] * (ln(rho[i]) - 1.0) for i in range(Nspec)])

    # Total energy density (per volume)
    rho_e = 0.5 * rho_tot * inner(u, u) + rho_F + theta * rho_s

    # Mobility M_{ij} and related fluxes
    def M_ij(i, j):
        return (rho[i] if i == j else 0.0) - rho[i] * rho[j] / rho_tot

    def grad_mu_over_theta(j):
        return grad(mu[j] / theta)

    # Skew-symmetric convection form C(rho u, v, w)
    def C_skw(rho_u, v_in, w_in):
        return 0.5 * (
            inner(dot(rho_u, nabla_grad(v_in)), w_in)
          - inner(dot(rho_u, nabla_grad(w_in)), v_in)
        )

    # Symmetric gradient
    Du = sym(grad(u))

    # Residual
    F = 0

    for i in range(Nspec):  # Mass (for each species)
        diff_flux_i = sum(M_ij(i, j) * grad_mu_over_theta(j) for j in range(Nspec))
        F += (
            inner(Dt(rho[i]), psi[i])
          - inner(rho[i] * u, grad(psi[i]))
          + inner(diff_flux_i, grad(psi[i]))
        ) * dx

    for i in range(Nspec):  # Chemical potential
        d_rho_e_d_rhoi = diff(rho_e, variable(rho[i]))
        F += (
            (mu[i] - d_rho_e_d_rhoi - V_i[i] * p) * zeta[i]
        ) * dx

    rhou = rho_tot * u  # Momentum
    F += (
        inner(sqrt_rho * Dt(m), v)
      + C_skw(rhou, u, v)
      + 2.0 * nu_c * inner(Du, grad(v))
      - inner(p, div(v))
      + sum([
            inner(rho[i] * grad(mu[i] - V_i[i] * p), v)
        for i in range(Nspec)])
      + inner(rho_s * grad(theta), v)
    ) * dx

    F += (  # Auxiliary momentum-like thing
        (inner(m, w) - inner(sqrt_rho * u, w))
    ) * dx

    F += (  # Pseudo-incompressibility
        div(u) * q
      + sum([sum([
            V_i[i] * inner(M_ij(i, j) * grad_mu_over_theta(j), grad(q))
        for j in range(Nspec)]) for i in range(Nspec)])
    ) * dx

    inv_theta = 1.0 / theta  # Entropy
    F += (
        inner(Dt(rho_s), omega)
      - inner(rho_s * u, grad(omega))
      - 2.0 * nu_c * inner(Du, grad(u)) * inv_theta * omega
      - K_c * inner(grad(inv_theta), grad(omega * inv_theta))
      - sum([sum([
            inner(grad_mu_over_theta(j), grad((omega * mu[i]) * inv_theta)) * M_ij(i, j)
        for j in range(Nspec)]) for i in range(Nspec)])
    ) * dx

    F += (  # Temperature
        (theta - diff(rho_e, variable(rho_s))) * gamma
    ) * dx

    # Time integrator
    t = Constant(0.0)
    sp = {
        # Example linear solver settings (tune as needed)
        "snes_monitor" : None,
        "snes_converged_reason" : None,
        "ksp_monitor" : None,
        "ksp_converged_reason" : None,
    }
    scheme_dict = {
        "cpg"   : ContinuousPetrovGalerkinScheme(time_deg, quadrature_degree=4*time_deg-1),  # Can up degree as needed
        "gauss" : GaussLegendre(time_deg)
    }
    stepper = TimeStepper(
        F, scheme_dict[scheme.lower()], t, dt_c, z,
        solver_parameters=sp
        # solver_parameters=sp, aux_indices=[Nspec+2+i for i in range(Nspec+3)]
    )

    # Initial conditions (Idk just trying this out)
    rho_ic = 0.6 + 0.2 * sin(2*pi*x) * sin(2*pi*y)
    rho_out[0].interpolate(rho_ic)
    for i in range(1, Nspec): rho_out[i].interpolate((1 - rho_ic)/(Nspec - 1))
    theta_out.interpolate(1.0)
    for i in range(Nspec): mu_out[i].interpolate(diff(rho_e, variable(rho[i])) + V_i[i] * p)

    # Set up outputs
    E_form = rho_e * dx
    S_form = rho_s * dx

    t_arr = []
    E_arr = []
    S_arr = []
    if write_qois:
        qoi_path = out_path / "qois.csv"
        with qoi_path.open("w", encoding="utf-8") as f:
            f.write("time,energy,entropy\n")

    def record_and_log():
        t_out = float(t)
        E_out = float(assemble(E_form))
        S_out = float(assemble(S_form))
        print(BLUE % f"Time (t) = {t_out:.6f}")
        print(GREEN % f"Energy  = {E_out:.8e}")
        print(GREEN % f"Entropy = {S_out:.8e}")
        t_arr.append(t_out)
        E_arr.append(E_out)
        S_arr.append(S_out)
        if write_qois:
            with (out_path / "qois.csv").open("a", encoding="utf-8") as f:
                f.write(f"{t_out},{E_out},{S_out}\n")

    record_and_log()

    if write_vtk:
        vtk = VTKFile(str(out_path / "u.pvd"))
        u_out.rename("Barycentric velocity (u)")
        vtk.write(u_out, time=float(t))

    # Time loop
    for _ in range(Nt):
        stepper.advance()
        t.assign(float(t) + float(dt_c))
        if write_vtk: vtk.write(u_out, time=float(t))
        record_and_log()

    return {"time": t_arr, "energy": E_arr, "entropy": S_arr}

In [93]:
_ = stefan_maxwell()

tsfc:WARNING Estimated quadrature degree 24 more than tenfold greater than any argument/coefficient degree (max 2)
tsfc:WARNING Estimated quadrature degree 24 more than tenfold greater than any argument/coefficient degree (max 2)
tsfc:WARNING Estimated quadrature degree 25 more than tenfold greater than any argument/coefficient degree (max 2)
tsfc:WARNING Estimated quadrature degree 24 more than tenfold greater than any argument/coefficient degree (max 2)
tsfc:WARNING Estimated quadrature degree 24 more than tenfold greater than any argument/coefficient degree (max 2)
tsfc:WARNING Estimated quadrature degree 24 more than tenfold greater than any argument/coefficient degree (max 2)
tsfc:WARNING Estimated quadrature degree 24 more than tenfold greater than any argument/coefficient degree (max 2)
tsfc:WARNING Estimated quadrature degree 24 more than tenfold greater than any argument/coefficient degree (max 2)
tsfc:WARNING Estimated quadrature degree 25 more than tenfold greater than any a

Time (t) = 0.000000
Energy  = -1.65226090e+00
Entropy = 0.00000000e+00


tsfc:WARNING Estimated quadrature degree 21 more than tenfold greater than any argument/coefficient degree (max 2)


  0 SNES Function norm 4.166666666667e-02
    Residual norms for petsctools_10_ solve.
    0 KSP Residual norm 4.166666666667e-02
    1 KSP Residual norm 2.741118691081e-16
    Linear petsctools_10_ solve converged due to CONVERGED_ITS iterations 1
  Nonlinear petsctools_10_ solve did not converge due to DIVERGED_FNORM_NAN iterations 0


ConvergenceError: Nonlinear solve failed to converge after 0 nonlinear iterations.
Reason:
   DIVERGED_FNORM_NAN